In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/user/project/python/rmsKit")
import os
import torch
from lattice import KH, save_npy
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import rms_torch
import numpy as np
import rms

# u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
u = np.load("/home/user/project/python/rmsKit/array/torch/KH/3site/smel/"+
        "Jx_1_Jy_1_Jz_1_hx_0_hz_0/e_100_lr_0.001/loss_7.985083/u/0.npy")
device = torch.device("cuda")
p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
h_list = KH.local("3site", p)
U = np.kron(u, u)

#* don't forget to add minus sign to h
h_list_prime = [-np.array(U @ h @ U.T) for h in h_list[0]]
save_npy(f"/home/user/project/python/rmsKit/array/torch/KH/3site/smel/"+
        "Jx_1_Jy_1_Jz_1_hx_0_hz_0/e_100_lr_0.001/loss_7.985083/H", h_list_prime)

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
H_gpu = torch.tensor(H, dtype=torch.float64, device=device)
model = rms_torch.UnitaryRieman(H.shape[0], 8, device=device, u0=u).to(device)
loss_smel = rms_torch.SystemMinimumEnergyLoss(H, device=device).to(device)
loss_sel = rms_torch.SystemEnergyLoss(H, device=device).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)


## Before optimization

In [23]:
loss_smel.initializer(torch.eye(4096, device=device, dtype = torch.float64))
sel = loss_sel(torch.eye(4096, device=device, dtype = torch.float64)).item()
smel = loss_smel(torch.eye(4096, device=device, dtype = torch.float64)).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.788146799451779
System minimum energy:  8.195037963688478


## SML optimization 

We optimized minimum eigen value

In [25]:
loss_smel.initializer(compiled_model())
sel = loss_sel(compiled_model()).item()
smel = loss_smel(compiled_model()).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.690365032623017
System minimum energy:  7.985082781440802


## SEL optimization


In [37]:
u1 = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
model2 = rms_torch.UnitaryRieman(H.shape[0], 8, device=device, u0=u1).to(device)
compiled_model2 = torch.compile(model2, dynamic = False, fullgraph=True)

loss_smel.initializer(compiled_model2())
sel = loss_sel(compiled_model2()).item()
smel = loss_smel(compiled_model2()).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.699970951642108
System minimum energy:  8.030657783532432


In [17]:
H = KH.system([2,2],"3site", p)
H_prime = np.kron(U, U) @ H @ np.kron(U, U).T


In [27]:
h1 = np.load("0.npy")
h2 = np.load("1.npy")
h3 = np.load("2.npy")

In [5]:
def int_to_custom_base(number, base, digits):
    custom_base_str = np.base_repr(number, base)  # Convert the integer to its custom base representation
    padded_custom_base_str = custom_base_str.zfill(digits)  # Pad with leading zeros to ensure the specified number of digits
    
    custom_base_vector = np.array([int(digit, base) for digit in padded_custom_base_str])
    return custom_base_vector

In [6]:
h_list_shift = []
for h in h_list_prime:
    h = h.copy()
    shift = 0
    for i in range(h.shape[0]):
        shift = min(shift, h[i, i])
    shift *= -1;
    shift += 0.1
    h += shift * np.eye(h.shape[0])
    h_list_shift.append(h)

In [12]:
np.diag(h_list_shift[0])

array([0.84999522, 0.76666207, 0.59999522, 0.68333736, 0.76667329,
       0.68332615, 0.85000422, 0.60000424, 1.01666653, 0.93333337,
       0.76666643, 0.34999972, 0.43333565, 0.84999748, 0.5166666 ,
       0.26666675, 0.34999989, 0.26666675, 0.1       , 0.68333272,
       0.76666865, 0.1833308 , 0.84999955, 0.59999945, 0.68333376,
       0.60000061, 0.43333376, 0.5166651 , 0.60000103, 0.51666468,
       0.68333196, 0.43333198, 0.51666881, 0.43333566, 0.26666892,
       0.85000192, 0.93333785, 0.34999972, 1.01666876, 0.76666866,
       0.68333514, 0.60000199, 0.43333515, 0.51665929, 0.59999522,
       0.51666607, 0.68332615, 0.43332617, 0.84999883, 0.76666568,
       0.59999884, 0.68333374, 0.76666967, 0.68332976, 0.8500006 ,
       0.60000062, 0.84999957, 0.76666642, 0.59999948, 0.18333302,
       0.26666895, 0.68333053, 0.3499999 , 0.10000005])

In [13]:
0.847224 * 6 / 5

1.0166688000000002

In [7]:
for j in range(8):
    max_val = 0
    for i in range(8**6):
        state = int_to_custom_base(i, 8, 6)
        sum_val = 0
        sum_val += np.diag(h_list_shift[0])[state[0] + j * 8]
        sum_val += np.diag(h_list_shift[1])[state[2] + j * 8]
        sum_val += np.diag(h_list_shift[2])[state[4] + j * 8]
        sum_val += np.diag(h_list_shift[0])[state[1] * 8 + j]
        sum_val += np.diag(h_list_shift[1])[state[3] * 8 + j]
        sum_val += np.diag(h_list_shift[2])[state[5] * 8 + j]
        max_val = max(max_val, sum_val)
        print(sum_val / 12, state)
    print(max_val/12)

0.4249998635934749 [0 0 0 0 0 0]
0.41805543417713054 [0 0 0 0 0 1]
0.4041665308010143 [0 0 0 0 0 2]
0.4111102086541014 [0 0 0 0 0 3]
0.4180548693357951 [0 0 0 0 0 4]
0.41111077435052246 [0 0 0 0 0 5]
0.4249991136805656 [0 0 0 0 0 6]
0.4041657820167978 [0 0 0 0 0 7]
0.4388883907748711 [0 0 0 0 1 0]
0.4319439613585267 [0 0 0 0 1 1]
0.41805505798241044 [0 0 0 0 1 2]
0.4249987358354976 [0 0 0 0 1 3]
0.43194339651719127 [0 0 0 0 1 4]
0.4249993015319186 [0 0 0 0 1 5]
0.43888764086196175 [0 0 0 0 1 6]
0.4180543091981939 [0 0 0 0 1 7]
0.3833328351430074 [0 0 0 0 2 0]
0.37638840572666304 [0 0 0 0 2 1]
0.3624995023505468 [0 0 0 0 2 2]
0.3694431802036339 [0 0 0 0 2 3]
0.3763878408853276 [0 0 0 0 2 4]
0.36944374590005497 [0 0 0 0 2 5]
0.3833320852300981 [0 0 0 0 2 6]
0.36249875356633027 [0 0 0 0 2 7]
0.41111051084073064 [0 0 0 0 3 0]
0.4041660814243862 [0 0 0 0 3 1]
0.39027717804826995 [0 0 0 0 3 2]
0.3972208559013571 [0 0 0 0 3 3]
0.40416551658305083 [0 0 0 0 3 4]
0.3972214215977781 [0 0 0 0 3 5]

KeyboardInterrupt: 

'max_val / 12'